#### Name: HARIPRASAD SIVAPATHAM ANAND
#### Student Number: 202205341
#### Email: x2022gmv@gmail.com
#### Accuracy Achieved: 77.775%

In [111]:
import numpy as np
import pandas as pd
from copy import deepcopy

In [112]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [113]:
def clean_data(data):
    cols = ["Age","Fare"]
    for col in cols:
        data[col].fillna(data[col].mean(),inplace=True)
    data.Embarked.fillna(data.Embarked.mode()[0],inplace=True)
    return data;

In [114]:
data = train.copy(deep=True)
data = clean_data(data)

In [115]:
# We will use this function to get perform feature engineering on name
def spliter(name):
    tokens = name.split(", ")
    ppl_kind = tokens[1].split('.')[0]
    
    return ppl_kind

In [116]:
# We will replace null with string in cabins for dataset to be fed in machine learning model
def cabin_letter(cabin):
    if str(cabin) == "nan":
        return "NA"
    else:
        return str(cabin)[0] # Letters

In [117]:
# Function to convert Age from number to numerical encoded data
def age_range(age):
    if age <=0:
        return 0 
    elif age > 20 and age <=40:
        return 1
    elif age >40 and age<=60:
        return 2
    else:
        return 3 

In [118]:
# Check if passenger is alone or with someone 
def is_alone(id_):
    if id_ in ids.values:
        return 1
    else:
        return 0

In [119]:
# Handle parch 
def is_with_nanny(id_):
    if id_ in ids.values:
        return 1
    else:
        return 0

In [123]:
# Handle Tickets
def isnum(ticket):
    if ticket.isnumeric():
        return 1
    else:
        return 0

In [124]:
def transform(data):
    global ids
    # Passenger loneliness. 
    ids = data[data.Parch + data.SibSp == 1]['PassengerId']
    data['is_alone'] = data.PassengerId.apply(is_alone)
    
    ids = data[(data.Age <= 15) & (data.Parch == 0)]['PassengerId']
    data['is_with_nanny'] = data.PassengerId.apply(is_with_nanny)
    
    # Handling the age column.
    data['Agerange'] = data.Age.apply(age_range)
    
    # Names titles
    data['Ppl_kind'] = data.Name.apply(spliter)
    replace = data['Ppl_kind'].unique()
    data.Ppl_kind.replace(to_replace=replace, value=list(range(len(replace))), inplace=True)
    
    # Cabins groups
    data['CabinGroup'] = data.Cabin.apply(cabin_letter)
    replace = data['CabinGroup'].unique()
    data.CabinGroup.replace(to_replace=replace, value=list(range(len(replace))), inplace=True)
    
    # Tickets types
    data['TicketType'] = data.Ticket.apply(isnum)
    
    # Changing the categorical columns to numeric
    data.Embarked = data.Embarked.map({'C':0, 'S':1, 'Q':2})
    data['Ppl_kind'] = data['Ppl_kind'].map({0:0, # zero
                                             1:1, 2:1, 3:1, # 1st
                                             4:2, 5:2, # 2nd
                                             6:3, 7:3, 8:3, 9:3, 10:3, 11:3, 12:3, 13:3, 14:3, 15:3, 16:3 # 3rd
                                            })
    data['Agerange'] = data['Agerange'].map({0:0, 1:1, 2:2, 3:2})
    data.Sex = data.Sex.map({'male':0, 'female':1})

    # Dropping useless columns 
    data.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'SibSp', 'Parch', 'Age'], axis=1, inplace=True)
    
    return data;


In [125]:
data = transform(data)

In [127]:
# Cleaning and transforming test data
nTest = clean_data(test)
nTest = transform(nTest)

In [128]:
# Droping non required columns 
data.drop(['TicketType', 'is_with_nanny'],axis=1,inplace=True)
nTest.drop(['TicketType', 'is_with_nanny'],axis=1,inplace=True)

In [129]:
# Model selection 
X = data.drop("Survived",axis=1)
y = data.Survived

In [130]:
# Scaling data 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
nTest_scaled = scaler.transform(nTest)

In [131]:
# Splitting into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

In [136]:
# Models to be used
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [137]:
# probability=True for SVC
algos = [
    RandomForestClassifier(),
    GaussianNB(),
    DecisionTreeClassifier(),
    KNeighborsClassifier(),
    LogisticRegression(),
    SVC(probability=True),
]

df = {"Model": [], "Accuracy Score in Train": [], "Accuracy Score in Test": [], "Cross Validation Mean": []}
for algo in algos:
    model = algo.fit(X_train, y_train)
    proba = algo.predict_proba(X_test)
    y_pred = []
    for p in proba:
        if p[0] > p[1]:
            y_pred.append(0)
        else:
            y_pred.append(1)
    df["Model"].append(str(algo)[:str(algo).find('(')])
    df["Accuracy Score in Test"].append(accuracy_score(y_test, y_pred))
    df["Accuracy Score in Train"].append(algo.score(X_train, y_train))
    df["Cross Validation Mean"].append(cross_val_score(algo, X_scaled, y, cv=10).mean())

models = pd.DataFrame(df)
models

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

,Model,Accuracy Score in Train,Accuracy Score in Test,Cross Validation Mean
0,RandomForestClassifier,0.948034,0.804469,0.815930
1,GaussianNB,0.771067,0.737430,0.750911
2,DecisionTreeClassifier,0.948034,0.798883,0.809201
3,KNeighborsClassifier,0.846910,0.782123,0.800337
4,LogisticRegression,0.796348,0.759777,0.782260
5,SVC,0.841292,0.793296,0.815968


In [145]:
%%capture --no-display
from sklearn.model_selection import GridSearchCV
n_estimators = [int(x) for x in np.linspace(start = 0, stop = 200, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
gsc = GridSearchCV(RandomForestClassifier(), random_grid)
gsc.fit(X_scaled, y)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False],
                         'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100,
                                       110, None],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [0, 22, 44, 66, 88, 111, 133, 155, 177,
                                          200]})

In [146]:
print(f"The best paramters: {gsc.best_params_}")
print(f"The best score: {gsc.best_score_}")


The best paramters: {'bootstrap': True, 'max_depth': 80, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 44}
The best score: 0.8384031134266525


In [151]:
cls = RandomForestClassifier(bootstrap=True,max_depth=80,max_features='sqrt',min_samples_leaf=2,min_samples_split=10,n_estimators=44).fit(X_scaled, y)

In [152]:
sub = pd.read_csv('./gender_submission.csv')
predicted = cls.predict(nTest_scaled)
sub.to_csv('submission_final_3.csv', index=False)